In [92]:
from edgar import set_identity

In [2]:
# must be set to comply with SEC regulations

set_identity("Neal Lockhart nhl3388@uncw.edu")

[11:47:39] INFO     Identity of the Edgar REST client set to [Neal   ]8;id=573781;file:///home/neal/nealsfiles/stock_advisor_model/venv/lib/python3.12/site-packages/edgar/core.py\core.py]8;;\:]8;id=39775;file:///home/neal/nealsfiles/stock_advisor_model/venv/lib/python3.12/site-packages/edgar/core.py#161\161]8;;\
                    Lockhart nhl3388@uncw.edu]                                  


## Filings: 10-k

### Markdown

In [22]:
from edgar import *

In [23]:
filings = Company("SNOW").get_filings(form="10-K").latest(2)

In [34]:
md = filings[0].markdown()

/home/neal/nealsfiles/stock_advisor_model/venv/lib/python3.12/site-packages/edgar/documents.py:429: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(html, features='lxml')


In [110]:
with open('markdown.md', 'w', encoding='utf-8') as f:
    chunk = 8192
    for i in range(0, len(md), chunk):
        f.write(md[i : i+chunk])

### HTML

In [28]:
html = filings[0].html()

In [69]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")

In [91]:
with open('html.txt', 'w') as f:
    f.write(soup.prettify())

In [79]:
risk_factors_header = soup.find(string="ITEM 1A. RISK FACTORS")

In [112]:
element = risk_factors_header.find_parent("div")

In [113]:
next_element = element.find_next_sibling()

In [114]:
element.get_text()

'ITEM 1A. RISK FACTORS'

In [115]:
next_element.get_text()

'Our operations and financial results are subject to various risks and uncertainties, including those described below. You should consider and read carefully all of the risks and uncertainties described below, together with all of the other information contained in this Annual Report on Form 10-K, including the sections titled “Special Note about Forward-Looking Statements” and “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and our consolidated financial statements and related notes, before making an investment decision. The risks described below are not the only ones we face. The occurrence of any of the following risks or additional risks and uncertainties not presently known to us or that we currently believe to be immaterial could materially and adversely affect our business, financial condition, results of operations, or growth prospects. In such case, the trading price of our common stock could decline. You should not interpret our disclos

In [116]:
next_element.find_next_sibling().get_text()

'Risks Related to Our Business and Operations'

In [107]:
def get_full_section_text(start_header):
    section_text = []
    current_element = start_header.find_parent("div")
    
    while current_element:
        next_element = current_element.find_next_sibling()
        
        # Check if we've hit the next major section
        if next_element and any(text.strip().startswith("ITEM ") 
                              for text in next_element.stripped_strings):
            break
            
        if next_element:
            section_text.append(next_element.get_text(strip=True))
            current_element = next_element
        else:
            break
    
    return ' '.join(section_text)

risk_factors_header = soup.find(string="ITEM 1A. RISK FACTORS")
full_risk_section = get_full_section_text(risk_factors_header)

In [109]:
with open('text.txt', 'w') as f:
    f.write(full_risk_section)